<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#加载数据和模型" data-toc-modified-id="加载数据和模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>加载数据和模型</a></span></li><li><span><a href="#精度，召回率，F和时间随参数的变化" data-toc-modified-id="精度，召回率，F和时间随参数的变化-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>精度，召回率，F和时间随参数的变化</a></span><ul class="toc-item"><li><span><a href="#随MAX_DIS变化" data-toc-modified-id="随MAX_DIS变化-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>随MAX_DIS变化</a></span></li><li><span><a href="#随MAX_ANGLE变化" data-toc-modified-id="随MAX_ANGLE变化-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>随MAX_ANGLE变化</a></span></li><li><span><a href="#随MIN_LEN变化" data-toc-modified-id="随MIN_LEN变化-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>随MIN_LEN变化</a></span></li><li><span><a href="#随MIN_COVER_ANGLE变化" data-toc-modified-id="随MIN_COVER_ANGLE变化-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>随MIN_COVER_ANGLE变化</a></span></li><li><span><a href="#随MIN_COVER_RHO1变化" data-toc-modified-id="随MIN_COVER_RHO1变化-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>随MIN_COVER_RHO1变化</a></span></li><li><span><a href="#随MIN_COVER_RHO2变化" data-toc-modified-id="随MIN_COVER_RHO2变化-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>随MIN_COVER_RHO2变化</a></span></li></ul></li></ul></div>

In [1]:
%matplotlib inline
raw_path = r'/home/liucc/Workspace/02_Ellipse/03_results/01_industry/01_raw/'
label_path = r'/home/liucc/Workspace/02_Ellipse/03_results/01_industry/04_ellipse_label/'

In [2]:
import os 
fnames = os.listdir(raw_path)
fnames = [n[:-4] for n in fnames if n[-3:]=='jpg']
fnames.sort()
print(fnames[:10])

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010']


# 加载数据和模型

In [3]:
import sys
sys.path.append('/home/liucc/Workspace/02_Ellipse/01_data')
from cc_process_image import *
import imageio

raws = get_all_images(f'{raw_path}',fnames,'jpg')

print(len(raws))

/home/liucc/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


66


In [4]:
gts = []

for name in fnames:
    lname = label_path+name+'.txt'
    
    with open(lname,'r') as f:
        txt = f.read()
        
        ells_t = [[float(v) for v in l.split(' ')] for l in txt.split('\n')]
        
        gts.append(ells_t)
        

print(len(gts))

66


# 精度，召回率，F和时间随参数的变化

In [5]:
import sys
sys.path.append('/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect')
from cc_ellipse_v2 import *
from experiment import *

In [8]:
import time
from skimage.feature import canny

def one_task(glob):
    start = time.time()

    prs = []
    
    for im in raws:
        im = preprocess_image(im)
        imo = canny(im, sigma=2)
        ells = detect_ellipses_im(imo,glob)
        
        prs.append(ells)

    end = time.time()
    
    costT = (end-start)/len(raws)
    
    precision,recall,F1,_,_ = cal_metrics(prs,gts)
    
    #print(precision,recall,F1,costT)
    
    return [precision,recall,F1,costT]

In [21]:
#全局参数
GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 30,
    'MIN_COVER_ANGLE' : 140,
    'MIN_COVER_RHO1' : 0.5,
    'MIN_COVER_RHO2' : 0.4,
}
one_task(GLOB)

[0.8186936936936937, 0.5126939351198871, 0.6305290546400694, 1.080109675725301]

## 随MAX_DIS变化

In [11]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 20,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.3,
}

res01 = []
xs01 = list(range(0,40,10))
for x in xs01:
    GLOB['MAX_DIS'] = x
    res = one_task(GLOB)
    res01.append(res)
    print(f'{x}:{res}')

0:[0.6544876886417792, 0.5811001410437235, 0.615614493836384, 1.1067602634429932]
10:[0.6534496431403648, 0.5811001410437235, 0.6151549085479657, 1.0687275221853545]
20:[0.6518987341772152, 0.5811001410437235, 0.6144668158090978, 1.0837179711370757]
30:[0.6486697965571205, 0.5846262341325811, 0.6149851632047477, 1.1035451347177678]


## 随MAX_ANGLE变化

In [12]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 20,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.3,
}

res02 = []
xs02 = list(range(30,90,10))
for x in xs02:
    GLOB['MAX_ANGLE'] = x
    res = one_task(GLOB)
    res02.append(res)
    print(f'{x}:{res}')

30:[0.6501597444089456, 0.5740479548660085, 0.6097378277153557, 1.115043408942945]
40:[0.6544876886417792, 0.5811001410437235, 0.615614493836384, 1.0913130261681296]
50:[0.6438035853468433, 0.5825105782792666, 0.6116253239540911, 1.075164845495513]
60:[0.6422513492675405, 0.5874471086036671, 0.6136279926335175, 0.9187812516183564]
70:[0.6311787072243346, 0.5853314527503526, 0.6073911452616172, 1.0762237743897871]
80:[0.6237772761474794, 0.5846262341325811, 0.6035675282125955, 1.1037329543720593]


## 随MIN_LEN变化

In [17]:
from matplotlib.font_manager import FontProperties
fp1 = FontProperties(family="DejaVu Sans",size=20)
fp2 = FontProperties(family="DejaVu Sans",size=18)

GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 30,
    'MIN_COVER_ANGLE' : 120,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.3,
}

res1 = []
xs1 = list(range(10,50,10))
for x in xs1:
    GLOB['MIN_LEN'] = x
    res = one_task(GLOB)
    res1.append(res)
    print(f'{x}:{res}')

10:[0.6334355828220859, 0.5825105782792666, 0.6069066862601029, 1.0141292774316035]
20:[0.6544876886417792, 0.5811001410437235, 0.615614493836384, 1.0820574182452578]
30:[0.7006255585344057, 0.5528913963328632, 0.6180528182893181, 1.0843325528231533]
40:[0.7303030303030303, 0.5098730606488011, 0.600498338870432, 1.0586144129435222]


## 随MIN_COVER_ANGLE变化

In [18]:
GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 30,
    'MIN_COVER_ANGLE' : 140,
    'MIN_COVER_RHO1' : 0.4,
    'MIN_COVER_RHO2' : 0.3,
}

res2 = []
xs2 = np.arange(100,181,20)
for x in xs2:
    GLOB['MIN_COVER_ANGLE'] = x
    res = one_task(GLOB)
    res2.append(res)
    print(f'{x}:{res}')

100:[0.6788445199660152, 0.5634696755994358, 0.6157996146435453, 1.0046431982156]
120:[0.7006255585344057, 0.5528913963328632, 0.6180528182893181, 1.1188142624768345]
140:[0.7311320754716981, 0.5465444287729196, 0.6255044390637612, 1.073564836473176]
160:[0.7524850894632207, 0.5338504936530324, 0.6245874587458745, 1.0872209758469553]
180:[0.7734457323498419, 0.5176304654442877, 0.6201943388255174, 1.0751049337965068]


## 随MIN_COVER_RHO1变化

In [19]:
GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 30,
    'MIN_COVER_ANGLE' : 140,
    'MIN_COVER_RHO1' : 0.5,
    'MIN_COVER_RHO2' : 0.3,
}

res3 = []
xs3 = np.arange(0.3,0.7,0.1)
for x in xs3:
    GLOB['MIN_COVER_RHO1'] = x
    res = one_task(GLOB)
    res3.append(res)
    print(f'{x}:{res}')

0.3:[0.7238183503243745, 0.5507757404795487, 0.6255506607929516, 1.0298127622315378]
0.4:[0.7311320754716981, 0.5465444287729196, 0.6255044390637612, 1.1222271666382297]
0.5:[0.7561942517343905, 0.5380818053596615, 0.6287597857437165, 1.0705036972508286]
0.6000000000000001:[0.7744034707158352, 0.5035260930888575, 0.6102564102564103, 1.0586770881306042]


## 随MIN_COVER_RHO2变化

In [20]:
GLOB = {
    'MAX_FIT_DIS':4,
    'MAX_DIS':0,
    'MAX_ANGLE':40,
    'MIN_LEN' : 30,
    'MIN_COVER_ANGLE' : 140,
    'MIN_COVER_RHO1' : 0.5,
    'MIN_COVER_RHO2' : 0.4,
}


res4 = []
xs4 = np.arange(0,0.6,0.1)
for x in xs4:
    GLOB['MIN_COVER_RHO2'] = x
    res = one_task(GLOB)
    res4.append(res)
    print(f'{x}:{res}')

0.0:[0.7037374658158614, 0.5444287729196051, 0.6139165009940358, 1.0573296727556172]
0.1:[0.7037374658158614, 0.5444287729196051, 0.6139165009940358, 1.0422324773037073]
0.2:[0.7076076993583869, 0.5444287729196051, 0.6153846153846154, 1.0571448730699944]
0.30000000000000004:[0.7561942517343905, 0.5380818053596615, 0.6287597857437165, 1.0543191324580798]
0.4:[0.8186936936936937, 0.5126939351198871, 0.6305290546400694, 1.0672889189286665]
0.5:[0.8860569715142429, 0.41678420310296194, 0.5669064748201439, 1.093539667851997]
